In [7]:
# Only need to do this once...
!pip install flask
!pip install --upgrade google-cloud-datastore


  Using cached https://files.pythonhosted.org/packages/b8/6c/bedcab39e8dc969f7a48d058dbacd69fc07ce3f817a03de875902016f667/google_cloud_datastore-1.15.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ad/fc/6e8c449185cb8862af353c1164100ff75e32d55ba1de3baf9eaa01b7d2a9/google_cloud_core-1.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0b/2f/1aa41b81d8ed16c22c02dbc7a1c9890dc81f826958063d7e2ae16f475a5e/google_auth-1.27.1-py2.py3-none-any.whl
pandas-gbq 0.3.0 has requirement google-cloud-bigquery>=0.28.0, but you'll have google-cloud-bigquery 0.23.0 which is incompatible.
google-cloud-bigquery 0.23.0 has requirement google-cloud-core<0.24dev,>=0.23.1, but you'll have google-cloud-core 1.6.0 which is incompatible.
tensorboard 1.8.0 has requirement bleach==1.5.0, but you'll have ble

In [8]:
import json
import re
from flask import Flask, request, jsonify, make_response

from google.cloud import datastore

In [ ]:
app = Flask(__name__)
results = ""
rsize = 0
from random import seed
from random import randint
seed(1)

@app.route('/webhook/', methods=['POST'])
def handle():
    req = request.get_json(silent=True, force=True)
    print 'Request:'
    print(json.dumps(req, indent=4))
    topic = req.get('queryResult').get('parameters').get('topic')
    
    print topic
    # there could be multiple topics in a sentence
    if isinstance(topic, list):
        topic = topic[0]
    #topic = re.sub(r'[^\w\s]', '', topic)
    topic = topic.lower()
    print("topic is ",topic)
    
    rsp = buildReply('Command not recognized')
    if (topic == "schedule"):
        rsp = scheduleMovie(req)
    elif (topic == "profile" or topic == "account"):
        rsp = getProfileAccount()
    elif (topic == "movie recommendation" ):
        rsp = getRecommendations()
        
    rsp = json.dumps(rsp, indent=4)
    print rsp
    r = make_response(rsp)
    r.headers['Content-Type'] = 'application/json'
    return r    
  
def scheduleMovie(req):
    confirm = req.get('queryResult').get('parameters').get('confirm').lower()
    print confirm
    if (confirm == "yes"):
        return buildReply('Sending request ...')
    return buildReply('Aborted')
  
def getProfileAccount():
    urls = ""
    with open('netflix_urls.txt') as netflix_urls:
        urls = netflix_urls.read()
    print urls
    return buildReply(urls)

def getRecommendations():
    global results 
    global rsize
    print "recom"
    if (rsize ==0):
        client = datastore.Client()
        query = client.query(kind='netflix')
        results = list(query.fetch(limit=100))
        rsize = len(results)
        print rsize
    return buildReply(results[randint(0, rsize -1 )]['title']+"\n"+results[randint(0, rsize-1)]['title'])
    
# below was the original function in the sample code, untouched. 
def getResponse(topic):
    
    client = datastore.Client()
    query = client.query(kind='Synonym')
    key = client.key('Synonym', topic)
    query.key_filter(key, '=')
    results = list(query.fetch())
    
    if len(results) == 0:
        return buildReply('I can\'t find that in the handbook...')
    
    print results[0]['synonym']
    
    query = client.query(kind='Topic')
    key = client.key('Topic', results[0]['synonym'])
    query.key_filter(key, '=')
    results = list(query.fetch())
    
    print results[0]['action_text']
    
    return buildReply(results[0]['action_text'])

def buildReply(info):
    return {
        'fulfillmentText': info,
    }

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


Request:
{
    "queryResult": {
        "fulfillmentMessages": [
            {
                "text": {
                    "text": [
                        ""
                    ]
                }
            }
        ], 
        "allRequiredParamsPresent": true, 
        "parameters": {
            "topic": [
                "movie recommendation"
            ]
        }, 
        "languageCode": "en", 
        "intentDetectionConfidence": 0.75147843, 
        "action": "lookup", 
        "intent": {
            "displayName": "Topic", 
            "name": "projects/i-backbone-307006/agent/intents/8a6ee9f0-16e4-43d3-88cc-8d6baf409556"
        }, 
        "queryText": "movie recommendation", 
        "outputContexts": [
            {
                "name": "projects/i-backbone-307006/agent/sessions/dfMessenger-7355229/contexts/__system_counters__", 
                "parameters": {
                    "topic": [
                        "movie recommendation"
                    ]

127.0.0.1 - - [14/Mar/2021 10:08:53] "POST /webhook/ HTTP/1.1" 200 -


100
{
    "fulfillmentText": "Brain Games\nBoy and the World"
}
